In [ ]:
!pip install opencv-python numpy scikit-image

In [ ]:
!pip install tensorflow

In [ ]:
pip install mahotas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 9.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#%matplotlib inline
import os
import cv2
from tensorflow import keras
import tensorflow as tf
#import keras
# example of converting an image with the Keras API
from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing.image import image_utils
from skimage.feature import graycomatrix, graycoprops
from keras.utils import load_img, img_to_array
from PIL import Image
#from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten,MaxPool2D
# from tensorflow.keras import layers, models
from keras.utils import to_categorical
from keras import layers, models
from sklearn.metrics import confusion_matrix , classification_report
import pickle
from skimage.feature import greycomatrix, graycoprops, local_binary_pattern, hog
import mahotas.features.texture as mhtex

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Main Dataset Loading Process
#training path
TRAIN_PATH = '/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/train/'
# testing path
TEST_PATH = '/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/test/'

In [ ]:
# Creating a directory to save the filtered images
output_dir_train = '/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Filter_train'
output_dir_test = '/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Filter_test'

os.makedirs(output_dir_train, exist_ok=True)
os.makedirs(output_dir_test, exist_ok=True)

In [ ]:
# divinding it in classes
classes = { 0:"carcinoma_in_situ", 1:"light_dysplastic", 2:"moderate_dysplastic",3:"normal_columnar",
            4:"normal_intermediate",5:"normal_superficiel",6:"severe_dysplastic"}

In [ ]:
for cl in classes:
  path = os.path.join(output_dir_train, classes[cl])
  os.makedirs(path, exist_ok=True)

for cl in classes:
  path = os.path.join(output_dir_test, classes[cl])
  os.makedirs(path, exist_ok=True)

In [ ]:
# List all image files in the dataset directory
image_files_train = [os.path.join(TRAIN_PATH, filename) for filename in os.listdir(TRAIN_PATH) if filename.lower().endswith('.BMP')]
image_files_test = [os.path.join(TEST_PATH, filename) for filename in os.listdir(TEST_PATH) if filename.lower().endswith('.BMP')]

# NLM Filter

In [ ]:
# Defination of NLM filter function
def nlm_filter(image):
    filtered_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)  # Adjust parameters as needed
    return filtered_image

In [ ]:
# Applying NLM filtering to each image, converting to PNG, and saving the filtered images

# Train Images
for cl in classes:
    pth = TRAIN_PATH + classes[cl]
    for image_filename in os.listdir(pth):
        # Load the image using OpenCV
        image = cv2.imread(pth + '/' + image_filename)

        # Convert the image to RGB (OpenCV reads images in BGR format)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply NLM filtering
        filtered_image = nlm_filter(image_rgb)

        # Get the file name without extension
        filename = os.path.splitext(os.path.basename(image_filename))[0]

        # Save the filtered image as PNG format
        output_file = os.path.join(output_dir_train + '/' + classes[cl], f'{filename}_nlmfiltered.png')
        cv2.imwrite(output_file, filtered_image)

print("Noise removal and PNG conversion completed for train images.")


# Test Images
for cl in classes:
    pth = TEST_PATH + classes[cl]
    for image_filename in os.listdir(pth):
        # Load the image using OpenCV
        image = cv2.imread(pth + '/' + image_filename)

        # Convert the image to RGB (OpenCV reads images in BGR format)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply NLM filtering
        filtered_image = nlm_filter(image_rgb)

        # Get the file name without extension
        filename = os.path.splitext(os.path.basename(image_filename))[0]

        # Save the filtered image as PNG format
        output_file = os.path.join(output_dir_test + '/' + classes[cl], f'{filename}_nlmfiltered.png')
        cv2.imwrite(output_file, filtered_image)

print("Noise removal and PNG conversion completed for test images.")


Noise removal and PNG conversion completed for train images.
Noise removal and PNG conversion completed for test images.


In [ ]:
# List all image files in the output directory
filtered_files_train = os.listdir(output_dir_train)
filtered_files_test = os.listdir(output_dir_test)

# Printing the filtered image filenames for train
print("Train classes:")
for file_name_train in filtered_files_train:
    print(file_name_train)
print("******************")
print("Test classes:")
# Printing the filtered image filenames for test
for file_name_test in filtered_files_test:
    print(file_name_test)

Train classes:
normal_superficiel
carcinoma_in_situ
light_dysplastic
moderate_dysplastic
normal_intermediate
normal_columnar
severe_dysplastic
******************
Test classes:
severe_dysplastic
normal_superficiel
moderate_dysplastic
carcinoma_in_situ
normal_columnar
normal_intermediate
light_dysplastic


# Augmentation

In [ ]:
# Train Folder for Augmentation
input_folder_train = '/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Filter_train'
output_folder_train = '/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Augmentation/Train'

for cl in classes:
  path = os.path.join(output_folder_train, classes[cl])
  os.makedirs(path, exist_ok=True)


#Create an ImageDataGenerator and specify the augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=45,      # Rotate the image between -45 to 45 degrees
    width_shift_range=0.2,  # Shift the width of the image by up to 20%
    height_shift_range=0.2, # Shift the height of the image by up to 20%
    shear_range=0.2,        # Apply shear transformation with a shear factor of 0.2
    zoom_range=0.2,         # Zoom the image by up to 20%
    horizontal_flip=True,   # Flip the image horizontally
    vertical_flip=True     # Flip the image vertically
    # Add more augmentation parameters here
)

In [ ]:
# Generate augmented images and save them to the output folder of train folder.
for cl in classes:
  pth = input_folder_train + '/' + classes[cl]
  for filename in os.listdir(pth):
    # Load the image using OpenCV
    image_path = cv2.imread(pth + '/' + filename)

    # Convert the image to a NumPy array
    image_array = img_to_array(image_path)

    # Reshape the image array to match the expected input shape of the generator
    image_array = image_array.reshape((1,) + image_array.shape)

    # Generate augmented images using the datagen.flow() method
    augmented_images = datagen.flow(
          image_array,
          batch_size=1,
          save_to_dir= os.path.join(output_folder_train + '/' + classes[cl]), # Save the augmented images to the output folder
          save_prefix='augmented',    # Prefix to use for the saved images
          save_format='png'           # Save images in PNG format
      )

    # Generate and save the augmented images
    num_augmented_images = 5  # Specify the number of augmented images to generate
    for i, augmented_image in enumerate(augmented_images):
        if i >= num_augmented_images:
            break

        # Convert the augmented image to PIL format
        augmented_image_pil = Image.fromarray(augmented_image[0].astype('uint8'))

        # Save the augmented image as PNG
        save_filename = f'{filename.split(".")[0]}_{i}.png'
        save_path = os.path.join(output_folder_train + '/' + classes[cl], save_filename)
        augmented_image_pil.save(save_path)
print("Augmented images done for training set")

Augmented images done for training set


In [ ]:
# Test Folder for Augmentation
input_folder_test = '/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Filter_test'
output_folder_test = '/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Augmentation/Test'

for cl in classes:
  path = os.path.join(output_folder_test, classes[cl])
  os.makedirs(path, exist_ok=True)

In [ ]:
# Generated augmented images and save them to the output folder of test folder.
for cl in classes:
  pth = input_folder_test + '/' + classes[cl]
  for filename in os.listdir(pth):
    # Load the image using OpenCV
    image_path = cv2.imread(pth + '/' + filename)

    # Convert the image to a NumPy array
    image_array = img_to_array(image_path)

    # Reshape the image array to match the expected input shape of the generator
    image_array = image_array.reshape((1,) + image_array.shape)

    # Generate augmented images using the datagen.flow() method
    augmented_images = datagen.flow(
          image_array,
          batch_size=1,
          save_to_dir= os.path.join(output_folder_test + '/' + classes[cl]), # Save the augmented images to the output folder
          save_prefix='augmented',    # Prefix to use for the saved images
          save_format='png'           # Save images in PNG format
      )

    # Generate and save the augmented images
    num_augmented_images = 5  # Specify the number of augmented images to generate
    for i, augmented_image in enumerate(augmented_images):
        if i >= num_augmented_images:
            break

        # Convert the augmented image to PIL format
        augmented_image_pil = Image.fromarray(augmented_image[0].astype('uint8'))

        # Save the augmented image as PNG
        save_filename = f'{filename.split(".")[0]}_{i}.png'
        save_path = os.path.join(output_folder_test + '/' + classes[cl], save_filename)
        augmented_image_pil.save(save_path)
print("Augmented images done for testing set")

Augmented images done for testing set


# Watershed Segmentation

In [ ]:
def segment_cell(image_path):
    # Reading the images
    image = cv2.imread(image_path)

    # Converting the images to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform adaptive thresholding to obtain a binary image
    _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Perform morphological operations to remove noise and fill gaps
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find the sure background region
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Find the sure foreground region
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding the unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Creating a marker image
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    # Applying the watershed algorithm
    markers = cv2.watershed(image, markers)

    # Applying Color the segmented cells
    image[markers == -1] = [0, 0, 255]  # Mark boundaries with red color

    return image

In [ ]:
# train folder
watershed_train = 'Herlev dataset/Watershed/Trainnn'
# test folder
watershed_test = 'Herlev dataset/Watershed/Testtt'

for cl in classes:
  path_train = os.path.join(watershed_train, classes[cl])
  path_test = os.path.join(watershed_test, classes[cl])
  os.makedirs(path_train, exist_ok=True)
  os.makedirs(path_test, exist_ok=True)

In [ ]:
import os

for cl in classes:
    # Provide the path to the folder containing the input images
    folder_path = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Augmentation/Train', classes[cl])
    watershed_testclass = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Watershed/Trainnn', classes[cl])

    # Create subfolders if they don't exist
    if not os.path.exists(watershed_testclass):
        os.makedirs(watershed_testclass)

    # Get a list of all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Process each image in the folder
    for image_file in image_files:
        # Construct the full path to the image
        image_path = os.path.join(folder_path, image_file)

        # Perform image segmentation
        segmented_image = segment_cell(image_path)

        # Save the segmented image with the same filename and PNG extension in the subfolder
        output_path = os.path.join(watershed_testclass, image_file)  # Use the same filename
        cv2.imwrite(output_path, segmented_image)

print("Segmented test images saved successfully as png")


Segmented test images saved successfully as png


In [ ]:
import os

for cl in classes:
    # Provide the path to the folder containing the input images
    folder_path = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Augmentation/Test', classes[cl])
    watershed_testclass = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Watershed/Testtt', classes[cl])

    # Create subfolders if they don't exist
    if not os.path.exists(watershed_testclass):
        os.makedirs(watershed_testclass)

    # Get a list of all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Process each image in the folder
    for image_file in image_files:
        # Construct the full path to the image
        image_path = os.path.join(folder_path, image_file)

        # Perform image segmentation
        segmented_image = segment_cell(image_path)

        # Save the segmented image with the same filename and PNG extension in the subfolder
        output_path = os.path.join(watershed_testclass, image_file)  # Use the same filename
        cv2.imwrite(output_path, segmented_image)

print("Segmented test images saved successfully as png")


Segmented test images saved successfully as png


# GLCM for Feature Extraction

In [ ]:
# GLCM for Training Images
for cl in classes:
  folder_path = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Watershed/Trainnn', classes[cl])

  for filename in os.listdir(folder_path):
    if filename.endswith('.png'):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path, 0)  # Read the image in grayscale

        distances = [1]
        angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]

        glcm = graycomatrix(image, distances, angles, levels=256, symmetric=True, normed=True)

        properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        features = np.ravel([graycoprops(glcm, prop).reshape(-1) for prop in properties])

        # Additional properties using mahotas
        haralick_features = mhtex.haralick(image)
        asm = haralick_features[:, 0].mean()
        cluster_prominence = haralick_features[:, 3].mean()
        cluster_shade = haralick_features[:, 4].mean()
        max_prob = haralick_features[:, 8].mean()

        # Additional features using skimage
        radius = 3
        n_points = 8 * radius
        lbp = local_binary_pattern(image, n_points, radius, method='uniform')
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        lbp_hist = lbp_hist.astype(np.float)
        lbp_hist /= (lbp_hist.sum() + 1e-6)  # Normalize LBP histogram

        hog_features = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1))

        print(f'Features for image: {filename}')
        for prop, feature in zip(properties + ['ASM', 'cluster_prominence', 'cluster_shade', 'max_prob'],
                                 features.tolist() + [asm, cluster_prominence, cluster_shade, max_prob]):
            print(f'{prop}: {feature}')
        print(f'LBP: {lbp_hist.tolist()}') #Local Binary Patterns
        print(f'HOG: {hog_features.tolist()}') #Histogram of Oriented Gradients
        print('------------------------------')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# GLCM for Testing Images
for cl in classes:
  folder_path = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Watershed/Testtt', classes[cl])

  for filename in os.listdir(folder_path):
    if filename.endswith('.png'):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path, 0)  # Read the image in grayscale

        distances = [1]
        angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]

        glcm = graycomatrix(image, distances, angles, levels=256, symmetric=True, normed=True)

        properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        features = np.ravel([graycoprops(glcm, prop).reshape(-1) for prop in properties])

        # Additional properties using mahotas
        haralick_features = mhtex.haralick(image)
        asm = haralick_features[:, 0].mean()
        cluster_prominence = haralick_features[:, 3].mean()
        cluster_shade = haralick_features[:, 4].mean()
        max_prob = haralick_features[:, 8].mean()

        # Additional features using skimage
        radius = 3
        n_points = 8 * radius
        lbp = local_binary_pattern(image, n_points, radius, method='uniform')
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        lbp_hist = lbp_hist.astype(np.float)
        lbp_hist /= (lbp_hist.sum() + 1e-6)  # Normalize LBP histogram

        hog_features = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1))

        print(f'Features for image: {filename}')
        for prop, feature in zip(properties + ['ASM', 'cluster_prominence', 'cluster_shade', 'max_prob'],
                                 features.tolist() + [asm, cluster_prominence, cluster_shade, max_prob]):
            print(f'{prop}: {feature}')
        print(f'LBP: {lbp_hist.tolist()}')
        print(f'HOG: {hog_features.tolist()}')
        print('------------------------------')

Output hidden; open in https://colab.research.google.com to view.

# PCA

In [ ]:
# PCA for training
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from matplotlib.pyplot import imread

# Assuming 'classes' is a dictionary mapping class labels to folder names

# Define the output folder for PCA-transformed images
output_folder = 'Herlev dataset/PCA_Transformed_Images/training'
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

for cl in classes:
    # Define the path to the folder containing the images
    folder_path = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Watershed/Trainnn', classes[cl])

    # Process each image in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            img = plt.imread(image_path).astype(np.float32) / 255.0  # Load and normalize image

            # Convert RGB image to grayscale (assuming it's an RGB image)
            img = img.mean(axis=2)

            plt.imshow(img, cmap="gray")
            plt.title("Original Image")
            plt.show()

            def transform_image(percentage, output_filename):
                percentage = percentage / 100
                tswizzle_pca = PCA(n_components=percentage).fit(img)
                transformed = tswizzle_pca.transform(img)
                projection = tswizzle_pca.inverse_transform(transformed)

                plt.imshow(projection, cmap="gray")
                plt.title(f"PCA Transformed Image (Percentage: {percentage * 100}%)")
                plt.savefig(output_filename)  # Save the PCA-transformed image
                plt.show()

            # Create the output filename for the PCA-transformed image
            output_filename = os.path.join(output_folder, f'{filename[:-4]}_pca.png')

            transform_image(95, output_filename)  # Call the transform function with desired percentage


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# PCA for testing
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Assuming 'classes' is a dictionary mapping class labels to folder names

# Define the output folder for PCA-transformed images
output_folder = 'Herlev dataset/PCA_Transformed_Images/testing'
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

for cl in classes:
    # Define the path to the folder containing the images
    folder_path = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Watershed/Testtt', classes[cl])

    # Process each image in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            img = plt.imread(image_path).astype(np.float32) / 255.0  # Load and normalize image

            # Convert RGB image to grayscale (assuming it's an RGB image)
            img = img.mean(axis=2)

            plt.imshow(img, cmap="gray")
            plt.title("Original Image")
            plt.show()

            def transform_image(percentage, output_filename):
                percentage = percentage / 100
                tswizzle_pca = PCA(n_components=percentage).fit(img)
                transformed = tswizzle_pca.transform(img)
                projection = tswizzle_pca.inverse_transform(transformed)

                plt.imshow(projection, cmap="gray")
                plt.title(f"PCA Transformed Image (Percentage: {percentage * 100}%)")
                plt.savefig(output_filename)  # Save the PCA-transformed image
                plt.show()

            # Create the output filename for the PCA-transformed image
            output_filename = os.path.join(output_folder, f'{filename[:-4]}_pca.png')

            transform_image(95, output_filename)  # Call the transform function with desired percentage


# Classification (Random Forest)

In [ ]:
                                                                          # OLD CODE
# # classification for training data
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Assuming 'classes' is a dictionary mapping class labels to folder names

# # Define the output folder for PCA-transformed images
# output_folder = 'PCA_Transformed_Images/training'
# os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

# X = []  # List to store PCA-transformed images as features
# y = []  # List to store corresponding class labels

# for cl in classes:
#     # Define the path to the folder containing the images
#     folder_path = os.path.join('Herlev dataset/Watershed/Trainnn', classes[cl])

#     # Process each image in the folder
#     for filename in os.listdir(folder_path):
#         if filename.endswith('.png'):
#             image_path = os.path.join(folder_path, filename)
#             img = plt.imread(image_path).astype(np.float32) / 255.0  # Load and normalize image

#             # Convert RGB image to grayscale (assuming it's an RGB image)
#             img = img.mean(axis=2)

#             # Apply PCA transformation
#             percentage = 89
#             tswizzle_pca = PCA(n_components=percentage / 100).fit(img)
#             transformed = tswizzle_pca.transform(img)

#             flattened_transformed = transformed.ravel()

#             # Check if the flattened_transformed has the expected length
#             expected_length = len(X[0]) if X else len(flattened_transformed)
#             if len(flattened_transformed) == expected_length:
#                 X.append(flattened_transformed)
#                 y.append(cl)  # Add class label

# # Convert the lists to arrays for training
# X = np.array(X)
# y = np.array(y)

# # Convert the lists to arrays for training
# X = np.array(X)
# y = np.array(y)

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train a Random Forest classifier
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_classifier.fit(X_train, y_train)

# # Predict using the trained classifier
# y_pred = rf_classifier.predict(X_test)

# # Evaluate the classifier
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)


Accuracy: 0.8888888888888888


In [ ]:
                                                                        # LATEST CODE

# classification for training data
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'classes' is a dictionary mapping class labels to folder names

# Define the output folder for PCA-transformed images
output_folder = 'PCA_Transformed_Images/training'
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

X = []  # List to store PCA-transformed images as features
y = []  # List to store corresponding class labels

for cl in classes:
    # Define the path to the folder containing the images
    folder_path = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Watershed/Trainnn', classes[cl])

    # Process each image in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            img = plt.imread(image_path).astype(np.float32) / 255.0  # Load and normalize image

            # Converting RGB image to grayscale
            img = img.mean(axis=2)

            # Applying PCA transformation
            percentage = 89
            tswizzle_pca = PCA(n_components=percentage / 100).fit(img)
            transformed = tswizzle_pca.transform(img)

            flattened_transformed = transformed.ravel()

            # Checking if the flattened_transformed has the expected length
            expected_length = len(X[0]) if X else len(flattened_transformed)
            if len(flattened_transformed) == expected_length:
                X.append(flattened_transformed)
                y.append(cl)  # Add class label

# Converting the lists to arrays for training
X = np.array(X)
y = np.array(y)

# Converting the lists to arrays for training
X = np.array(X)
y = np.array(y)

# Spliting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Predicting using the trained classifier
y_pred = rf_classifier.predict(X_test)

# Evaluating the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6


In [ ]:
                                                                          # OLD CODE
# # classification for testing data
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Assuming 'classes' is a dictionary mapping class labels to folder names

# # Define the output folder for PCA-transformed images
# output_folder = 'PCA_Transformed_Images/testing'
# os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

# X = []  # List to store PCA-transformed images as features
# y = []  # List to store corresponding class labels

# for cl in classes:
#     # Define the path to the folder containing the images
#     folder_path = os.path.join('Herlev dataset/Watershed/Testtt', classes[cl])

#     # Process each image in the folder
#     for filename in os.listdir(folder_path):
#         if filename.endswith('.png'):
#             image_path = os.path.join(folder_path, filename)
#             img = plt.imread(image_path).astype(np.float32) / 255.0  # Load and normalize image

#             # Convert RGB image to grayscale (assuming it's an RGB image)
#             img = img.mean(axis=2)

#             # Apply PCA transformation
#             percentage = 89
#             tswizzle_pca = PCA(n_components=percentage / 100).fit(img)
#             transformed = tswizzle_pca.transform(img)

#             flattened_transformed = transformed.ravel()

#             # Check if the flattened_transformed has the expected length
#             expected_length = len(X[0]) if X else len(flattened_transformed)
#             if len(flattened_transformed) == expected_length:
#                 X.append(flattened_transformed)
#                 y.append(cl)  # Add class label

# # Convert the lists to arrays for training
# X = np.array(X)
# y = np.array(y)

# # Convert the lists to arrays for training
# X = np.array(X)
# y = np.array(y)

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train a Random Forest classifier
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_classifier.fit(X_train, y_train)

# # Predict using the trained classifier
# y_pred = rf_classifier.predict(X_test)

# # Evaluate the classifier
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)


Accuracy: 1.0


In [ ]:
                                                                        # LATEST CODE

# classification for testing data
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'classes' is a dictionary mapping class labels to folder names

# Define the output folder for PCA-transformed images
output_folder = 'PCA_Transformed_Images/testing'
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

X = []  # List to store PCA-transformed images as features
y = []  # List to store corresponding class labels

for cl in classes:
    # Define the path to the folder containing the images
    folder_path = os.path.join('/content/drive/MyDrive/NEW 7th Sem Project/Herlev dataset/Watershed/Testtt', classes[cl])

    # Process each image in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            img = plt.imread(image_path).astype(np.float32) / 255.0  # Load and normalize image

            # Convert RGB image to grayscale (assuming it's an RGB image)
            img = img.mean(axis=2)

            # Apply PCA transformation
            percentage = 89
            tswizzle_pca = PCA(n_components=percentage / 100).fit(img)
            transformed = tswizzle_pca.transform(img)

            flattened_transformed = transformed.ravel()

            # Check if the flattened_transformed has the expected length
            expected_length = len(X[0]) if X else len(flattened_transformed)
            if len(flattened_transformed) == expected_length:
                X.append(flattened_transformed)
                y.append(cl)  # Add class label

# Convert the lists to arrays for training
X = np.array(X)
y = np.array(y)

# Convert the lists to arrays for training
X = np.array(X)
y = np.array(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Predict using the trained classifier
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.3333333333333333
